<a href="https://colab.research.google.com/github/FrancescoMK/Sentiment-Analysis-for-Stock-Market-Prediction/blob/master/Sentiment_Analysis_for_Price_Movement_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis for Price Movement Prediction**


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

In [ ]:
from fastai.text import *
from fastai.callbacks import *

### **Data Manipulation**

**Getting the data**

In [ ]:
path = f'data/'

*dowloading the data*

In [ ]:
!wget https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz -P {path}

*extracting the data*

In [ ]:
!tar -zxf {path}/'ag_news_csv.tgz' -C {path}

**Preparing the data**

In [ ]:
path = Path('data/ag_news_csv')
path.ls()

*explore and name .csv files*

In [ ]:
df_train = pd.read_csv(path/'train.csv',  names=["class", "title", "description"])
df_train.head()

In [ ]:
df_test = pd.read_csv(path/'test.csv',  names=["class", "title", "description"])
df_test.head()

*take the union of df_train and df_test*

In [ ]:
df = df_train.append(df_test, ignore_index=True)

In [ ]:
len(df), len(df_train), len(df_test)

In [ ]:
df.head()

### **Concatenated Language Model**

**Create data object with the data block API**

*convert words into numbers in steps: tokenization and numericalization*

In [ ]:
data_lm = (TextList.from_df(df, path, cols=['title', 'description'])
                .split_by_rand_pct(0.1)
                .label_for_lm()
                .databunch())

*save and load data object*

In [ ]:
data_lm.save('data_lm')

In [ ]:
data_lm = load_data(path, 'data_lm')

*show data object after tokenisation and numericalisation*

In [ ]:
len(data_lm.train_ds), len(data_lm.valid_ds)

In [ ]:
data_lm.show_batch()

*datasets are now represented through tokenized and numericalised text*

In [ ]:
data_lm.train_ds[100][0]

*correspondance from ids to tokens is stored in the vocab*
*attribute of the datasets, in a dictionary called itos*

In [ ]:
data_lm.vocab.itos[:10]

In [ ]:
data_lm.batch_size

**Pretrained model on wiki data**


*The pretrained model is downloaded by specifying the arch=AWD_LSTM and*

*pretrained=True, so fastai automatically downloads the pretrained model*

In [ ]:
learn = language_model_learner(data_lm, arch=AWD_LSTM,  drop_mult=0)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(skip_end=15)

*saving the best model after every epoch*

In [ ]:
learn.callback_fns.append(partial(SaveModelCallback, name='lm-stage-1'))

**Fit model**

In [ ]:
learn.fit_one_cycle(3, 3.98E-02)

In [ ]:
learn.load('lm-stage-1');

*fit model over 7 epochs*

In [ ]:
# learn.callback_fns.pop()
# learn.callback_fns.append(partial(SaveModelCallback, name='lm-stage-2'))

In [ ]:
# learn.fit_one_cycle(7, 3.98E-02)

In [ ]:
# learn.load('lm-stage-2');

**Unfreeze and fine tune all the layers**

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

*fit the fine tuned model*

In [ ]:
# learn.callback_fns.pop()
learn.callback_fns.append(partial(SaveModelCallback, name='lm-unfreeze-1'))

In [ ]:
learn.fit_one_cycle(3, 3e-04)

**Predict and Save**

*make prediction out of model*

In [ ]:
learn.load('lm-unfreeze-1');

In [ ]:
TEXT = "This year is going to challenge"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

*save the model and its encoder responsible* 

*for creating and updating the hidden state*

In [ ]:
learn.save_encoder('fine_tuned_enc')

### **Sentiment Classifier**

**Create a new data object**

In [ ]:
path = untar_data(URLs.IMDB)

*only grabs the labelled data and keeps those labels*

In [ ]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=bs))

data_clas.save('data_clas.pkl')

In [ ]:
data_clas = load_data(path, 'data_clas.pkl', bs=bs)

In [ ]:
data_clas.show_batch()

**Model Creation**

*create a model to classify those reviews and load the encoder*

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

*first fit*

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

In [ ]:
learn.save('first')

In [ ]:
learn.load('first');

*second fit*

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.save('second')

In [ ]:
learn.load('second');

*third fit*

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.save('third')

In [ ]:
learn.load('third');

**Prediction**

*unfreeze and fit*

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
learn.save('unfrozen')

In [ ]:
learn.load('unfrozen')

*make prediction*

In [ ]:
learn.predict("I really loved that movie, it was awesome!");

**Export Model for Reuters**

In [ ]:
learn.export(file = 'export_clas.pkl')

### **Apply to Stocks (Draft)**

**Get Reuters Data**

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
data_to_load = files.upload()

In [ ]:
import io

df_predict = pd.read_csv(io.BytesIO(uploaded['Reuters_Cleaned_V2.csv']), names=["ID", "Source", "Text"])
df_predict.head()

**Apply Model to Data**

In [ ]:
learn = load_learner(df_predict, file = 'export_clas.pkl')

In [ ]:
learn.data.add_test